In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.6 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 69.1 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tens

In [8]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
from PIL import Image, ImageOps
import gradio as gr
from tensorflow.keras import backend as K
from keras.saving import register_keras_serializable

# --- Constants ---
im_height = 256
im_width = 256
smooth = 100
labels = ['glioma_tumor', 'no_tumor', 'meningioma_tumor', 'pituitary_tumor']

# --- Custom Losses and Metrics ---
@register_keras_serializable()
def dice_loss(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))

def dice_coef(y_true, y_pred):
    y_truef = K.flatten(y_true)
    y_predf = K.flatten(y_pred)
    And = K.sum(y_truef * y_predf)
    return ((2 * And + smooth) / (K.sum(y_truef) + K.sum(y_predf) + smooth))

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def iou(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

def jac_distance(y_true, y_pred):
    y_truef = K.flatten(y_true)
    y_predf = K.flatten(y_pred)
    return -iou(y_true, y_pred)

# --- Model Loading ---
def load_classification_model():
    model = keras.models.load_model('/content/effnet (1).h5')
    return model

def load_segmentation_model():
    custom_objects = {
        'dice_loss': dice_loss,
        'dice_coef': dice_coef,
        'dice_coef_loss': dice_coef_loss,
        'iou': iou,
        'jac_distance': jac_distance,
    }
    model_seg = keras.models.load_model('/content/final_model.keras', custom_objects=custom_objects)
    return model_seg

# Load models once
model = load_classification_model()
model_seg = load_segmentation_model()

# --- Prediction Functions ---
def upload_predict(image, model):
    image_np = np.asarray(image)
    img_resize = cv2.resize(image_np, dsize=(150, 150), interpolation=cv2.INTER_CUBIC)
    img_reshape = np.expand_dims(img_resize, axis=0)
    pred = model.predict(img_reshape)
    pred_class = np.argmax(pred)
    confidence = np.max(pred)
    return pred_class, confidence

def add_grid(image, grid_size=50):
    h, w = image.shape[:2]
    for i in range(0, h, grid_size):
        cv2.line(image, (0, i), (w, i), (128, 128, 128), 1)
    for i in range(0, w, grid_size):
        cv2.line(image, (i, 0), (i, h), (128, 128, 128), 1)
    return image

def process_image(image):
    pred_class, confidence = upload_predict(image, model)
    classification_text = f"The image is classified as: {labels[pred_class]} with confidence {confidence:.2f}"

    image2 = np.asarray(image)
    original_display = cv2.resize(image2, (im_height, im_width))
    original_display = add_grid(original_display)

    if labels[pred_class] == 'no_tumor':
        return classification_text, [
            (original_display, "Original Image"),
            (original_display, "No Tumor Detected - Segmentation Skipped")
        ]

    img_seg = cv2.resize(image2, (im_height, im_width))
    img_norm = img_seg / 255.0
    input_seg = np.expand_dims(img_norm, axis=0)
    pred_seg = model_seg.predict(input_seg)

    pred_mask = (np.squeeze(pred_seg) > 0.5).astype(np.uint8)

    num_labels, labels_im = cv2.connectedComponents(pred_mask)
    largest_component = np.zeros_like(pred_mask)
    if num_labels > 1:
        max_area = 0
        max_label = 0
        for label in range(1, num_labels):
            area = np.sum(labels_im == label)
            if area > max_area:
                max_area = area
                max_label = label
        largest_component[labels_im == max_label] = 1
    else:
        largest_component = pred_mask

    alpha = 0.5
    color = np.array([0, 255, 255], dtype=np.uint8)
    overlay = np.zeros_like(img_seg, dtype=np.uint8)
    overlay[largest_component == 1] = color
    blended_img = cv2.addWeighted(img_seg, 1 - alpha, overlay, alpha, 0)
    blended_img = add_grid(blended_img)

    return classification_text, [
        (original_display, "Original Image"),
        (blended_img, "Segmentation Overlay (Largest Region)")
    ]

# --- Gradio UI ---
iface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="pil", label="Upload the image to be classified"),
    outputs=[
        gr.Textbox(label="Classification Result"),
        gr.Gallery(label="Original & Segmentation")
    ],
    title="Brain Tumor Classification & Segmentation",
    description=("Upload a brain MRI image to classify the tumor type and view a two-panel visualization. "
                 "The 'Original Image' shows the input MRI, and 'Segmentation Overlay' combines the original "
                 "image with a yellow semi-transparent tumor segmentation.")
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b510da2048fb3794c7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
